# 1. Import bibliotek

In [10]:

import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve


# 2. Wczytanie danych


In [11]:


# Wczytaj plik CSV (zmień nazwę pliku na właściwą dla Twojego zbioru)
df = pd.read_csv("zbior_3.csv")

pd.set_option('display.max_columns', 220) 

constant_cols = [c for c in df.columns if df[c].nunique() <= 1]
print("Constant columns:", constant_cols)


print("Rozmiar: ")
print(df.shape)
print("Podlgąd: ")
print(df.head())


Constant columns: ['szczegolnaFormaPrawna_Symbol', 'Zobowiazania_dostaw_uslug_pow12m_kapitale']
Rozmiar: 
(3000, 220)
Podlgąd: 
   szczegolnaFormaPrawna_Symbol  formaWlasnosci_Symbol  pkdKod  \
0                           117                    215    4941   
1                           117                    216    1629   
2                           117                    214    8559   
3                           117                    214    8010   
4                           117                    215    3511   

   wsk_liczba_dni_istnienia      Aktywa  Aktywa_trwale  \
0                      6866    55422.65       10027.40   
1                      9305   133689.19      106992.20   
2                      1285        0.00           0.00   
3                      3719  1693357.58       19839.43   
4                      1586     2859.66           0.00   

   Wartosci_niematerialne_prawne  Wartosc_firmy  Rzeczowe_aktywa_trwale  \
0                            0.0            0.0    

## 2.1 Usuwanie kolumn ze stałą wartością

In [12]:
constant_cols = [c for c in df.columns if df[c].nunique() <= 1]
print("Constant columns:", constant_cols)

print("Dropping constant columns")
df = df.drop(columns=constant_cols)

Constant columns: ['szczegolnaFormaPrawna_Symbol', 'Zobowiazania_dostaw_uslug_pow12m_kapitale']
Dropping constant columns


## 2.2 Usuwanie kulumn ze zduplikowanymi wartościami

In [13]:
dup_cols = df.columns[df.T.duplicated()]
print("Duplicate columns:", dup_cols.tolist())

print("Dropping duplicate columns")
df = df.drop(columns=dup_cols)

Duplicate columns: ['wsk_rent_sprzedazy', 'wsk_zysk_operacyjny_zob_1', 'wsk_zadluzenie_gotowka', 'wsk_ebitda_zobowiazan_odsetki_1', 'wsk_ebitda_zobowiazan_odsetki_2', 'wsk_poziom_kosztow_operacyjnych', 'wsk_poziom_kosztow_finansowych', 'wsk_struktura_kap_wlasnego_1']
Dropping duplicate columns


## 2.3 Usunięcie danych z ponad 90% brakujących wartości

In [14]:

missing_rate = df.isna().mean()
high_missing = missing_rate[missing_rate > 0.9].index
print("Highly missing columns:", high_missing.to_list())

print("Dropping highly missing columns")
df = df.drop(columns=high_missing)

Highly missing columns: ['RP_przeplywy_operacyjne', 'RP_amortyzacja', 'wsk_akt_generowania_got_2', 'wsk_pokrycie_wyd_fin_gotowkowe_2', 'wsk_pokrycie_zob_kr_gotowkowe_2', 'wsk_zadluzenia_gotowki_2', 'wsk_zysk_CF_operacyjny']
Dropping highly missing columns


## 2.4 Usunięcie danych z małą wariancją

In [15]:
# Example: df is your DataFrame
threshold = 0.75  # 85% of rows have the same value → drop

# Compute frequency ratio for each column
dominance = df.apply(lambda x: x.value_counts(normalize=True, dropna=False).iloc[0])

# List of columns to drop
low_var_cols = dominance[dominance > threshold].index.tolist()

low_var_cols = [c for c in low_var_cols if c != "default"] # Exclude "default"

print(f"Columns with >{threshold*100:.0f}% identical values:", low_var_cols)

print(f"Dropping {len(low_var_cols)} columns with low variance")
df = df.drop(columns=low_var_cols)

Columns with >75% identical values: ['Wartosci_niematerialne_prawne', 'Wartosc_firmy', 'Naleznosci_dlugoterminowe', 'Inwestycje_dlugoterminowe', 'Rozliczenia_miedzyokresowe_dlugie', 'Naleznosci_dostaw_uslug_12m_powiazane', 'Naleznosci_dostaw_uslug_pow12m_powiazane', 'Naleznosci_dostaw_uslug_12m_kapitale', 'Naleznosci_dostaw_uslug_pow12m_kapitale', 'Naleznosci_dostaw_uslug_pow12m_pozostale', 'Naleznosci_dostaw_uslug_pozostale_sadowe', 'Rezerwy_na_zobowiazania', 'Rezerwa_z_tytulu_odroczonego_podatku_dochodowego', 'Rezerwa_na_swiadczenia_emerytalne', 'Rezerwa_na_swiadczenia_emerytalne_dlugie', 'Rezerwa_na_swiadczenia_emerytalne_krotkie', 'Pozostale_rezerwy', 'Pozostale_rezerwy_krotkie', 'Zobowiazania_dlugoterminowe', 'Kredyty_pozyczki_dlugie', 'Inne_zobowiazania_fin_dlugoterminowe', 'Zobowiazania_dostaw_uslug_12m_powiazane', 'Zobowiazania_dostaw_uslug_pow12m_powiazane', 'Zobowiazania_dostaw_uslug_12m_kapitale', 'Kredyty_pozyczki_krotkie', 'Inne_zobowiazania_fin_krotkoterminowe', 'Zobowiaz

## 2.5 Usunięcie kolumn numerycznych z wysoką korelacją (zachłanny)


In [16]:
threshold_corr = 0.80  # 80% korelacji

# Wybierz tylko kolumny numeryczne (bez targetu)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != "default"]

 # Oblicz macierz korelacji
corr_matrix = df[numeric_cols].corr().abs()

# Algorytm zachłanny: iteruj przez kolumny i usuwaj skorelowane
cols_to_drop = []

for i in range(len(numeric_cols)):
    if numeric_cols[i] in cols_to_drop:
        continue
        
    for j in range(i + 1, len(numeric_cols)):
        if numeric_cols[j] in cols_to_drop:
            continue
        
        corr_value = corr_matrix.iloc[i, j]
        
        # Sprawdź czy korelacja nie jest NaN i przekracza próg
        if pd.notna(corr_value) and corr_value > threshold_corr:
            cols_to_drop.append(numeric_cols[j])
            print(f"Dropping {numeric_cols[j]} (corr={corr_value:.3f} with {numeric_cols[i]})")

print(f"\nTotal: dropping {len(cols_to_drop)} highly correlated numeric columns")
df = df.drop(columns=cols_to_drop)

Dropping Aktywa_obrotowe (corr=0.807 with Aktywa)
Dropping Naleznosci_krotkoterminowe (corr=0.804 with Aktywa)
Dropping Srodki_pieniezne (corr=0.809 with Aktywa)
Dropping Zobowiazania_rezerwy (corr=0.820 with Aktywa)
Dropping Zobowiazania_krotkoterminowe (corr=0.810 with Aktywa)
Dropping przychody_sprzedazy (corr=0.804 with Aktywa)
Dropping koszty_operacyjne (corr=0.803 with Aktywa)
Dropping amortyzacja (corr=0.805 with Aktywa)
Dropping przychody (corr=0.808 with Aktywa)
Dropping wsk_amortyzacja (corr=0.806 with Aktywa)
Dropping wsk_koszty_operacyjne (corr=0.805 with Aktywa)
Dropping wsk_Zobowiazania_krotkoterminowe (corr=0.811 with Aktywa)
Dropping Kapital_wlasny (corr=0.988 with Aktywa_trwale)
Dropping Kapital_podstawowy (corr=0.949 with Aktywa_trwale)
Dropping Srodki_trwale (corr=0.975 with Rzeczowe_aktywa_trwale)
Dropping wsk_rotacja_zapasow (corr=0.908 with Zapasy)
Dropping wsk_cykl_operacyjny (corr=0.890 with Zapasy)
Dropping wsk_cykl_konwersji_gotowki (corr=0.856 with Zapasy)
Dr

## 2.6 Usuniecie inf

In [17]:
# Sprawdź ile wartości inf jest w danych
inf_count = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
print(f"Liczba wartości inf w danych: {inf_count}")

if inf_count > 0:
    # Pokaż kolumny z inf
    inf_cols = df.select_dtypes(include=[np.number]).columns[np.isinf(df.select_dtypes(include=[np.number])).any()].tolist()
    print(f"Kolumny z wartościami inf: {inf_cols}")
    
    # Zamień inf na NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    print("Zamieniono wartości inf na NaN")
    
else:
    print("Brak wartości inf w danych")

Liczba wartości inf w danych: 1033
Kolumny z wartościami inf: ['wsk_kapital_do_aktywa', 'wsk_plynnosc_biez_1', 'wsk_struktura_kap_wlasnego_s_2']
Zamieniono wartości inf na NaN


## 2.7 Imputacja brakujących wartości

In [19]:
# Imputacja brakujących wartości: mediana dla numerycznych, moda dla kategorycznych
for col in df.columns:
    if df[col].isna().sum() > 0:  # Sprawdź czy są braki
        if df[col].dtype in ['int64', 'float64']:  # Kolumny numeryczne
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f"Filled {col} (numeric) with median: {median_value}")
        else:  # Kolumny kategoryczne
            mode_value = df[col].mode()[0]
            df[col].fillna(mode_value, inplace=True)
            print(f"Filled {col} (categorical) with mode: {mode_value}")

print(f"\nBrakujące wartości po imputacji: {df.isna().sum().sum()}")


Brakujące wartości po imputacji: 0


# 3. Eksport do CSV

In [20]:

df.to_csv("zbior_3_clean.csv", index=False)